In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from time import time
import pickle

# dataset path check
dataset_path = './cut_chars'
if not os.path.exists(dataset_path):
    print(f"Wrong Path: {dataset_path}")
else:
    print(f"Correct Path: {dataset_path}")

data = []
labels = []

Correct Path: ./cut_chars


In [2]:
# Collect labels
for root, dirs, files in os.walk(dataset_path):
    folder_name = os.path.basename(root)
    #print(f"Processing folder: {folder_name}, Number of files: {len(files)}")
    
    # skip empty files
    if len(files) == 0:
        continue
    
    # use folder names as the labels
    label = folder_name
    
    # fit label for every images
    for filename in files:
        filepath = os.path.join(root, filename) # join the path

        
        # grayscale processing
        image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        
        # resize image
        image = cv2.resize(image, (28, 28))

        # Image Binarisation
        image = np.where(image >= 75, 1, 0)
        
        # save the features and labels as the dataset
        data.append(image)
        labels.append(label)

In [3]:
# convert data type to numpy
data = np.array(data) 
labels = np.array(labels) 
print(f"Data shape: {data.shape}")
print(f"Labels shape: {labels.shape}")
print(labels)

Data shape: (51893, 28, 28)
Labels shape: (51893,)
['0' '0' '0' ... 'Z' 'Z' 'Z']


In [4]:
# encode the labels, create dictionary 
chars = [
    '0','1','2','3','4','5','6','7','8','9',
    'A','B','C','D','E','F','G','H','J','K','L','M','N','P','Q','R','S','T','U','V','W','X','Y','Z',
    '藏','川','鄂','甘','赣','贵','桂','黑','沪','吉','冀','津','晋','京','辽','鲁','蒙','闽','宁',
    '青','琼','陕','苏','皖','湘','新','渝','豫','粤','云','浙'
]

# chars to numbers
chars_mapping = {char: idx for idx, char in enumerate(chars)}
for char, idx in chars_mapping.items():
    print(f"'{char}': {idx}")

encoded_labels = [chars_mapping[label] for label in labels]
encoded_labels = np.array(encoded_labels)

# numbers to chars
inverse_chars_mapping = {idx: char for char, idx in chars_mapping.items()}
print(encoded_labels)


'0': 0
'1': 1
'2': 2
'3': 3
'4': 4
'5': 5
'6': 6
'7': 7
'8': 8
'9': 9
'A': 10
'B': 11
'C': 12
'D': 13
'E': 14
'F': 15
'G': 16
'H': 17
'J': 18
'K': 19
'L': 20
'M': 21
'N': 22
'P': 23
'Q': 24
'R': 25
'S': 26
'T': 27
'U': 28
'V': 29
'W': 30
'X': 31
'Y': 32
'Z': 33
'藏': 34
'川': 35
'鄂': 36
'甘': 37
'赣': 38
'贵': 39
'桂': 40
'黑': 41
'沪': 42
'吉': 43
'冀': 44
'津': 45
'晋': 46
'京': 47
'辽': 48
'鲁': 49
'蒙': 50
'闽': 51
'宁': 52
'青': 53
'琼': 54
'陕': 55
'苏': 56
'皖': 57
'湘': 58
'新': 59
'渝': 60
'豫': 61
'粤': 62
'云': 63
'浙': 64
[ 0  0  0 ... 33 33 33]


In [5]:
# train_test dataset split
X_train, X_test, Y_train, Y_test = train_test_split(data, encoded_labels, test_size=0.2, random_state=42, shuffle=True, stratify=labels)


In [6]:
# Train on GPU
#from PyTsetlinMachineCUDA.tm import MultiClassConvolutionalTsetlinMachine2D
#tm = MultiClassConvolutionalTsetlinMachine2D(500, 100, 5.0, (5, 5), max_weight=10)

# Trian on CPU
from pyTsetlinMachine.tm import MultiClassConvolutionalTsetlinMachine2D
tm = MultiClassConvolutionalTsetlinMachine2D(600, 300, 5.0, (10, 10), weighted_clauses=True)



In [7]:
for i in range(10):
    start = time()
    tm.fit(X_train, Y_train, epochs=1, incremental=True)
    stop = time()
    print("#%d Time: %.2fs" % (i+1, stop-start))   
    #result = 100*(tm.predict(X_test) == Y_test).mean()
    #print("#%d Accuracy: %.2f%% (%.2fs)" % (i+1, result, stop-start))
result = 100*(tm.predict(X_test) == Y_test).mean()
print("#%d Accuracy: %.2f%% (%.2fs)" % (i+1, result, stop-start))

#1 Time: 63.78s
#2 Time: 65.58s
#3 Time: 65.81s
#4 Time: 65.97s
#5 Time: 65.90s
#6 Time: 66.03s
#7 Time: 66.16s
#8 Time: 66.28s
#9 Time: 66.32s
#10 Time: 66.34s
#10 Accuracy: 97.33% (66.34s)


In [8]:
# save and load test
with open('TM.pkl', 'wb') as f:
    pickle.dump(tm, f)

with open('TM.pkl', 'rb') as f:
    tm = pickle.load(f)



In [9]:
# load test image
image_path = './test_char/t9.png'

# same preprocessing as training
sample_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
sample_image = cv2.resize(sample_image, (28, 28))
sample_image = np.where(sample_image >= 75, 1, 0)
#import matplotlib.pyplot as plt
#plt.imshow(sample_image, cmap='gray')

# prediction
predicted_label = tm.predict(sample_image) 
print(predicted_label)

# numbers to chars
predicted_char = inverse_chars_mapping[predicted_label[0]]
print(f'Predicted character: {predicted_char}')


[62 18  7  2 18 20 18 18 18 20 18 18 18 20 18 20 18 20 18 20 18 20 18 20
 18 18 20 18]
Predicted character: 粤
